In [117]:
import json
import pandas as pd
import glob
import datetime
from datetime import date, timedelta
import openpyxl
import pathlib
import numpy as np
from datetime import datetime
import os
import re
from pathlib import Path
import os.path
import time
from collections import OrderedDict

In [118]:
def convert_datetime(a):
    date_formats = ["%Y-%m-%d %H:%M:%S.%f", 
                    "%Y-%m-%d %H:%M:%S", 
                    "%m/%d/%Y %H:%M",
                    "%m-%d-%Y %H:%M", 
                    "%m/%d/%Y", 
                    "%Y-%m-%d", 
                    "%H:%M.%f",
                    "%m/%d/%Y %H:%M:%S"] 
    for fmt in date_formats:
        try:
            return datetime.strptime(str(a), fmt).strftime("%Y-%m-%d")
        except ValueError:
            pass
    try:
        return (datetime(1899, 12, 30) + timedelta(days=float(a))).strftime("%Y-%m-%d")
    except ValueError:
        pass
    if not re.match("\d+:\d+:\d+(\.\d+)?", str(a)):
        return str(a)
    return  

In [119]:
#Supporting Dashboard

#Variable
Date_variable = pd.read_excel('e:/tt/qa_daily_automation/variable.xlsx')
Date_variable

,Unnamed: 0,Start date,End date
0,appeal_diff_case,2023-08-27 00:00:00,2023-08-27
1,appeal_dailyreport,2023-08-19 00:00:00,2023-08-26
2,regular_diff_case,2023-08-21,2023-08-27
3,regular_false_case,2023-08-26 00:00:00,2023-08-26
4,rock_dailyreport,2023-08-19 00:00:00,2023-08-26
5,PCR_diff_case,2023-08-21,2023-08-27
6,PCR_false_case,2023-08-26 00:00:00,2023-08-26
7,sideprj_shoptab,2023-08-27 00:00:00,2023-08-27
8,sideprj_ugads,2023-08-27 00:00:00,2023-08-27
9,sideprj_dailyreport,2023-08-19 00:00:00,2023-08-26


In [120]:
#Supporting Dashboard

#Variable
Date_variable = pd.read_excel('E:/tt/qa_daily_automation/variable.xlsx')
from datetime import datetime
Date_variable['Start date'] = Date_variable['Start date'].apply(convert_datetime)
Date_variable['End date'] = Date_variable['End date'].apply(convert_datetime)

Start_Date_False_Regular = datetime.strptime(str(Date_variable.loc[3, 'Start date']), '%Y-%m-%d').strftime('%Y-%m-%d')
End_Date_False_Regular = datetime.strptime(str(Date_variable.loc[3, 'End date']), '%Y-%m-%d').strftime('%Y-%m-%d')

Start_Date_Diff_Regular = datetime.strptime(str(Date_variable.loc[2, 'Start date']), '%Y-%m-%d').strftime('%Y-%m-%d')
End_Date_Diff_Regular = datetime.strptime(str(Date_variable.loc[2, 'End date']), '%Y-%m-%d').strftime('%Y-%m-%d')

#Queue
queue_list = pd.read_excel('E:/tt/qa_daily_automation/Support_dashboard.xlsx',sheet_name='Queue List').values
queue_list = pd.DataFrame(queue_list)
headers = ['BUSINESS','COMPOUND','CATEGORY','MOD QUEUE ID','MOD QUEUE NAME',
           'QA QUEUE ID','QA QUEUE NAME','QUEUE GROUP','STATUS','Priority','Latency','AHT (secs)','Policy_type','Classify']
queue_list.columns = headers


In [121]:
Date_variable

,Unnamed: 0,Start date,End date
0,appeal_diff_case,2023-08-27,2023-08-27
1,appeal_dailyreport,2023-08-19,2023-08-26
2,regular_diff_case,2023-08-21,2023-08-27
3,regular_false_case,2023-08-26,2023-08-26
4,rock_dailyreport,2023-08-19,2023-08-26
5,PCR_diff_case,2023-08-21,2023-08-27
6,PCR_false_case,2023-08-26,2023-08-26
7,sideprj_shoptab,2023-08-27,2023-08-27
8,sideprj_ugads,2023-08-27,2023-08-27
9,sideprj_dailyreport,2023-08-19,2023-08-26


In [122]:
def convert_datetime(a):
    my_format="%Y-%m-%d %H:%M:%S"
    try:
        return datetime.strptime(str(a), "%Y-%m-%d %H:%M:%S").strftime(my_format)
    except:
        return datetime.strptime(str(a), "%m/%d/%Y %H:%M").strftime(my_format)
def convert_date(a):
    my_format="%Y-%m-%d"
    try:
        return datetime.strptime(str(a), "%Y-%m-%d %H:%M:%S").strftime(my_format)
    except:
        return datetime.strptime(str(a), "%m/%d/%Y %H:%M").strftime(my_format)
    

In [123]:
def convert_to_datetime(struct_time):
    return datetime(*struct_time[:6])

def input_data(data_dir):
    list_files = []

    for filename in pathlib.Path(data_dir).glob('**/*.*'):
        file_suffixes = filename.suffixes
        if file_suffixes[-1].lower() in ['.xlsx', '.csv']:
            if file_suffixes[-1].lower() == '.xlsx':
                export_time = os.path.getmtime(filename)
                export_time_datetime = convert_to_datetime(time.localtime(export_time))

                filename_with_export_time = filename.with_suffix('.xlsx')

                dfs = pd.read_excel(filename, sheet_name=None, skiprows=0, na_values=None, dtype={'Moderation Queue ID':object,'Object ID':object,'Task id':object,'Sampling Task ID':object})
                for sheet_name, df in dfs.items():
                    df['sheet_name'] = sheet_name
                    df['Export time'] = export_time_datetime
                    list_files.append(df)

            elif file_suffixes[-1].lower() == '.csv':
                export_time = os.path.getmtime(filename)
                export_time_datetime = convert_to_datetime(time.localtime(export_time))

                df = pd.read_csv(filename, skiprows=0, na_values=None, dtype={'Moderation Queue ID':object,'Object ID':object,'Task id':object,'Sampling Task ID':object,'task_id':object})
                df['sheet_name'] = 'CSV'
                df['Export time'] = export_time_datetime
                list_files.append(df)

    df_list = pd.concat(list_files, axis=0, ignore_index=True)
    return df_list

In [124]:
Rock_append = input_data('E:/tt/qa_daily_automation/Rock daily')
Rock_append = Rock_append.sort_values(by=['Export time'],ascending=True)
Rock_append = Rock_append.drop_duplicates(subset='Sampling Task ID',keep='last')

Rock_append = Rock_append.dropna(subset=['Moderation Time.1'])

In [125]:
grouped_df = Rock_append.groupby('Sampling Queue').first().reset_index()
columns = Rock_append.columns
new_df = grouped_df[columns]
new_df = new_df[['Sampling Queue','Sampling Task ID','Initial Moderation Result','Sampling respondent Result','Correct Moderated Results']]

In [126]:
#xử lý process_result
data_process_result = Rock_append[['Sampling Task ID','process result','process result.1','process result.2','process result.3','process result.4','process result.5']]

data_process_result['process result.1']=data_process_result['process result.1'].astype(str).replace('---',np.nan)
data_process_result['process result.2']=data_process_result['process result.2'].astype(str).replace('---',np.nan)
data_process_result['process result.3']=data_process_result['process result.3'].astype(str).replace('---',np.nan)
data_process_result['process result.4']=data_process_result['process result.4'].astype(str).replace('---',np.nan)
data_process_result['process result.5']=data_process_result['process result.5'].astype(str).replace('---',np.nan)

data_process_result['Sampling Task ID'] = data_process_result['Sampling Task ID'].astype('int64').astype(str)
data_process_result.set_index('Sampling Task ID',inplace=True)

def r(row):
    last_index = row.last_valid_index()
    return row[last_index] if last_index else np.nan

data_process_result['Final process result'] = data_process_result.apply(r, axis=1)
data_process_result.reset_index('Sampling Task ID',inplace=True)
data_process_result = data_process_result[['Sampling Task ID','Final process result']]
data_process_result['Final process result']=data_process_result['Final process result'].astype(str).replace('give up appealing ',"Failed appeal").astype(str)
data_process_result

C:\Users\anh.td\AppData\Local\Temp\ipykernel_17440\1526918364.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_process_result['process result.1']=data_process_result['process result.1'].astype(str).replace('---',np.nan)
C:\Users\anh.td\AppData\Local\Temp\ipykernel_17440\1526918364.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_process_result['process result.2']=data_process_result['process result.2'].astype(str).replace('---',np.nan)
C:\Users\anh.td\AppData\Local\Temp\ipykernel_17440\15269

,Sampling Task ID,Final process result
0,7258388699056964101,Failed appeal
1,7258722422264824326,Failed appeal
2,7258750765303448069,Failed appeal
3,7258350221615268358,Failed appeal
4,7258350224140386822,Failed appeal
...,...,...
4867,7269879005090693633,Failed appeal
4868,7269879122933858818,Failed appeal
4869,7269879190231515649,Failed appeal
4870,7270966570694787586,Failed appeal


In [127]:
Rock_append.columns

Index(['Selected duration', 'Claimant queue', 'Sampling Queue',
       'Operate status', 'Moderation Queue ID', 'Object ID', 'Task id',
       'Sampling Task ID', 'Task link', 'Moderation Round',
       'Initial Moderation Result', 'Latest Operator', 'Last operator role',
       'Specific operation', 'Moderation Time', 'Sampling respondent Result',
       'Moderation Time.1', 'Arbitrator's Result', 'Final Arbitrator's Result',
       'Corrector's result', 'Correct Moderated Results', 'Appealing Result',
       'Error towards', 'the one who is dealing now', 'Existing tags',
       'tagger', 'claimant', 'process result', 'process reason',
       'beginning time', 'Terminal time', 'handling time', 'Validator',
       'process result.1', 'process reason.1', 'beginning time.1',
       'Terminal time.1', 'handling time.1', 'Arbitrator', 'process result.2',
       'process reason.2', 'beginning time.2', 'Terminal time.2',
       'handling time.2', 'Respondent', 'process result.3', 'process re

In [128]:
Rock_append[['Moderated Results','Moderation Result','Respondent Result']] = Rock_append[['Correct Moderated Results','Initial Moderation Result','Sampling respondent Result']].replace(to_replace=r'^auditStatus: ',value= "Audit Result:: ",regex=True).astype(str)
pcr_result = ['remove', 'normal', 'selfvisible']
columns = ['Moderated Results', 'Moderation Result', 'Respondent Result']

def match_and_concatenate(cell):
    cell = str(cell).lower()
    cell = cell.replace('_x000d_', '')
    found_words = []
    audit_exp = re.search(r'(.*auditreason:)(.*)(\n|$)', cell)
    reject_exp = re.search(r'(.*rejectlabel:)(.*)(\n|$)', cell)
    if audit_exp:
        cell_split = audit_exp.group(2).strip()
    elif reject_exp:
        cell_split = reject_exp.group(2).strip()
    else:
        cell_split = cell
    cell_split = re.sub(r'[\[\]"]', '', cell_split)
    matches = re.findall('|'.join(pcr_result), cell_split, re.IGNORECASE)
    found_words.extend(matches)
    return ",".join(found_words) if found_words else None
for column in ['Moderated Results', 'Moderation Result', 'Respondent Result']:
    Rock_append.loc[Rock_append['Sampling Queue'].isin(['QA VN LL Product Comment', 'QA VN LL Product Comment Report']), column] = Rock_append.loc[Rock_append['Sampling Queue'].isin(['QA VN LL Product Comment', 'QA VN LL Product Comment Report']), column].apply(match_and_concatenate)

In [130]:
# Rock_append[(Rock_append['Sampling Queue'].isin(['QA VN LL Product Comment', 'QA VN LL Product Comment Report']))][['Object ID', 'Moderated Results', 'Moderation Result', 'Respondent Result']]

In [131]:
# Rock_append[(Rock_append['Sampling Queue'].isin(['QA VN LL Product Comment', 'QA VN LL Product Comment Report'])) & (Rock_append['Respondent Result'].isnull())][['Object ID', 'Moderated Results', 'Moderation Result', 'Respondent Result']]

In [132]:
Rock_append['Correct Moderated Results'] = Rock_append['Correct Moderated Results'].str.lower()
Rock_append['Sampling respondent Result'] = Rock_append['Sampling respondent Result'].str.lower()
Rock_append['Initial Moderation Result'] = Rock_append['Initial Moderation Result'].str.lower()

def process_cell(cell, aliases):
    results = []
    audit_results = re.findall(r'audit result: (.+)', cell, re.IGNORECASE)
    for audit_result in audit_results:
        for k, v in aliases.items():
            pat = '|'.join(v)
            if re.search(pat, audit_result, re.IGNORECASE):
                results.append(k)
    return ', '.join(results)

aliases = {
    "Not Approve": ['"status":400','":400,"','400.0','disapproval', 'disapproved','not approval','not approve' ,'notapproval','disapprove','not approved'],
    "Approve": ['approve','"status":200' ,'approval', 'approved','200' ,'200.0']
}

Rock_append['Final Result'] = Rock_append['Correct Moderated Results'].apply(lambda x: process_cell(str(x), aliases))
Rock_append['QA Result'] = Rock_append['Sampling respondent Result'].apply(lambda x: process_cell(str(x), aliases))
Rock_append['Mod Result'] = Rock_append['Initial Moderation Result'].apply(lambda x: process_cell(str(x), aliases))

Rock_append.loc[Rock_append['Final Result'] == '', 'Final Result'] = Rock_append.loc[Rock_append['Final Result'] == '', 'Moderated Results']
Rock_append.loc[Rock_append['QA Result'] == '', 'QA Result'] = Rock_append.loc[Rock_append['QA Result'] == '', 'Respondent Result']
Rock_append.loc[Rock_append['Mod Result'] == '', 'Mod Result'] = Rock_append.loc[Rock_append['Mod Result'] == '', 'Moderation Result']

def process_cell_2(cell):
    if cell is None:
        return '---'
    elif 'Not' in cell:
        return 'Not Approve'
    elif any(substring in cell for substring in ['normal', 'remove', 'selfvisible']):
        return cell
    else:
        return 'Approve'

Rock_append['Final Result'] = Rock_append['Final Result'].apply(process_cell_2)
Rock_append['QA Result'] = Rock_append['QA Result'].apply(process_cell_2)
Rock_append['Mod Result'] = Rock_append['Mod Result'].apply(process_cell_2)

Rock_append[Rock_append['Sampling Queue']=='QA VN LL Pigeon User Profile'][['Mod Result','QA Result','Final Result']]

,Mod Result,QA Result,Final Result
17384,Approve,Not Approve,Not Approve
17383,Approve,Not Approve,Not Approve


In [133]:
Rock_append['Correct Moderated Results'] = Rock_append['Correct Moderated Results'].str.lower()
Rock_append['Sampling respondent Result'] = Rock_append['Sampling respondent Result'].str.lower()
Rock_append['Initial Moderation Result'] = Rock_append['Initial Moderation Result'].str.lower()

Policy_errors_1 = [
    'prohibited products','unsupported products','restricted products','adult nudity and sexual activities','hateful behaviors','minor safety','absolute terms',
    'malicious comparison','title spam','abnormal price','inconsistent information','other languages','misleading functionality and effect','redirect traffic',
    'missing brand authorization','unfair trading','obscuring product images','target to minors','underage commercial behavior','false promise','false promises','sympathy promotion',
    'misleading size/weight/quantity','misleading patent/endorsement/metrics/awards','misleading origin','misleading return & refund service','misleading guarantees and warranties',
    'irrelevant promotion','give away','reproduced content','soliciting mutual interactions','still frame','possible pirated contents','possible pirated content',
    'non product promotion content','vulgar behavior','disturbing animal body','insulting and abusive content','body impurities','dirty environment','eye-catching sensitive content',
    'bind a sell b - prohibited product','misleading logistics','misleading brand','brand circumvention','copycat products','violence',
    'illegal behavior','hatespeech and harassment','possible ip infringement','spam','invalid company name','inaccurate company name','inaccurate company address','missing identity document',
    'expired documentation','black-and-white copy','unclear documentation','invalid watermark or logo','incomplete documentation','inaccurate representative name','inaccurate id number',
    'inaccurate documentation','inaccurate tax name','inaccurate tax number','inaccurate brand name','inaccurate trademark name','inaccurate trademark number','inaccurate application date',
    'inaccurate expiry date','invalid trademark','inaccurate registrant','unqualified destination market','inaccurate class(es) of goods/service','unqualified owner','inaccurate trademark certificate',
    'expired trademark certificate','unclear trademark certificate','inauthentic trademark certificate','unqualified 1st level distributor','inaccurate 1st level distributor letter',
    'expired 1st level distributor letter','unclear 1st level distributor letter','inauthentic 1st level distributor letter','unqualified 2nd level distributor',
    'inaccurate 2nd level distributor letter','expired 2nd level distributor letter','unclear 2nd level distributor letter','inauthentic 2nd level distributor letter',
    'unqualified third level distributor','inaccurate third level distributor letter','expired third level distributor letter','unclear third level distributor letter',
    'inauthentic third level distributor letter','missing buyer information','missing quantity of purchase item','missing purchase item information','missing purchase date',
    'unable to locate brand item product','purchase date mismatch','seller information mismatch','missing price of purchase item','missing seller information',
    'product quantity insufficient','exceeded validity period','altered document','lack of supportive information','unable to recognise source of purchase','underage applicant',
    'gibberish shop name','abnormal company status','traffic redirection','inaccurate company number','oversea personal seller','ps tampering trace','inauthentic documentation',
    'violent extremism','hate speech','unauthorized brand information','impersonation','suicide and dangerous acts','illegal activities and regulated goods',
    'harassment and bullying','child endangerment information','counterfeit product information','inauthentic certification of incorporation','inaccurate register date','erotic nudity','hateful contexts','partner name or logo relates to ''tiktok''','inauthentic work environment','repeated image','inappropriate watermark or logo',
    'incomplete information','inaccurate and misleading review','slightly vulgar','inaccurate package weight information','inappropriate speech','malicious behaviors','vulgar issues',
    'imagery depicting skin defects','imagery depicting indecent body organs','imagery depicting indecent teeth&oral','imagery depicting animal meat/carcasses',
    'imagery depicting unpleasant animal or insect','imagery depicting impurities','horror content','scene of squalor','offensive content','human excrement','mental illness content',
    'unsuitable for recommendation','surprise-based product','vat number check','slide show','counterfeit word indication','brand never produced','bind a sell b-prohibited product',
    'pirated content- dedup recall','pirated content','possible counterfeit product','click bait','click-bait','non shippable products','physical trademark infringement product','incorrect category high risk','low cost content',  
    'gambling related behaviour','information trademark infringement product','physically damaged id','suspicious brand promotion','non interactive content',
    ]

Policy_errors_2 = list(map(lambda x: x.replace(" ", "_") + "0005", Policy_errors_1))

Policy_errors = Policy_errors_1 + Policy_errors_2

cols_to_check = [ 'Initial Moderation Result','Sampling respondent Result','Correct Moderated Results']
cols_to_return = ['Mod Reason', 'QA Reason', 'Correct Policy']

queue_policy_type = queue_list[['QA QUEUE NAME','Policy_type']]
   
Rock_append = pd.merge(Rock_append,queue_policy_type, how='inner', left_on='Sampling Queue', right_on='QA QUEUE NAME')

for col_idx, col in enumerate(cols_to_check):
    Rock_append[cols_to_return[col_idx]] = ''
    for i, row in Rock_append.iterrows():
        cell_value = row[col]
        policy_type = row['Policy_type']
        if isinstance(cell_value, str):
            if 'form_info' in cell_value:
                form_info_index = cell_value.index('form_info')
                cell_value = cell_value[form_info_index:]
                match_list_1 = [match for match in Policy_errors_1 if match in cell_value]
                match_list_2 = [match for match in Policy_errors_2 if match in cell_value]
                if match_list_1 or match_list_2:
                    match_list = list(OrderedDict.fromkeys(match_list_1 + match_list_2))
                    final_reason = ", ".join(match_list).strip()
                    Rock_append.at[i, cols_to_return[col_idx]] = final_reason
            else:
                if policy_type == 'Multi-choice':
                    match_list_1 = [match for match in Policy_errors_1 if match in cell_value]
                    match_list_2 = [match for match in Policy_errors_2 if match in cell_value]
                    if match_list_1 or match_list_2:
                        match_list = list(OrderedDict.fromkeys(match_list_1 + match_list_2))
                        final_reason = ", ".join(match_list).strip()
                        Rock_append.at[i, cols_to_return[col_idx]] = final_reason
                elif policy_type == 'Single-choice':
                    match_list_1 = []
                    for match in Policy_errors_1:
                        if match in cell_value:
                            match_list_1.append(match)
                            break
                    if match_list_1:
                        final_reason = match_list_1[0]
                        Rock_append.at[i, cols_to_return[col_idx]] = final_reason
        else:
            continue


Rock_append['Final policy errors'] = Rock_append[['Mod Reason', 'QA Reason', 'Correct Policy']].apply(lambda x: x[x != ''].loc[x[x != ''].last_valid_index()] if x[x != ''].last_valid_index() else '', axis=1)

Rock_append['Final policy errors'] = Rock_append['Final policy errors'].replace("0005","",regex=True).replace("_"," ",regex=True).replace("counterfeit abnorma price product","abnormal price")
Rock_append['QA Reason'] = Rock_append['QA Reason'].replace("0005","",regex=True).replace("_"," ",regex=True).replace("counterfeit abnorma price product","abnormal price")
Rock_append['Mod Reason'] = Rock_append['Mod Reason'].replace("0005","",regex=True).replace("_"," ",regex=True).replace("counterfeit abnorma price product","abnormal price")

Rock_append['Final policy errors'] = Rock_append['Final policy errors'].apply(lambda x: ", ".join(set(x.split(", "))))
Rock_append['QA Reason'] = Rock_append['QA Reason'].apply(lambda x: ", ".join(set(x.split(", "))))
Rock_append['Mod Reason'] = Rock_append['Mod Reason'].apply(lambda x: ", ".join(set(x.split(", "))))

In [134]:
Rock_append[['Mod Reason','QA Reason','Correct Policy']].isna().sum()

Mod Reason        0
QA Reason         0
Correct Policy    0
dtype: int64

In [135]:
# Splitting the RCA labels
try:
    Rock_append[['RCA lvl1','RCA lvl2','RCA lvl3']] = Rock_append['Existing tags'].str.split(' > ', expand=True, n=2)
except:
    Rock_append[['RCA lvl1','RCA lvl2','RCA lvl3']] = np.nan

In [136]:
#Add diff case
Rock_append.loc[Rock_append['Mod Result'] != Rock_append['QA Result'],'Diff case']="FALSE"
Rock_append.loc[Rock_append['Diff case'].isnull(),'Diff case']="TRUE"

In [137]:
#Convert_time
Rock_append['Moderation Time'] = Rock_append['Moderation Time'].apply(convert_datetime)
Rock_append['Moderation Time'] = pd.to_datetime(Rock_append['Moderation Time'])

Rock_append['Moderation Time.1'] = Rock_append['Moderation Time.1'].apply(convert_datetime)
Rock_append['Moderation Time.1'] = pd.to_datetime(Rock_append['Moderation Time.1'])

In [138]:
Final_Rock_append = Rock_append.copy()
Final_Rock_append['Sampling Task ID'] = Final_Rock_append['Sampling Task ID'].astype('int64').astype(str)
Final_Rock_append.shape

(4872, 89)

In [139]:
Final_Rock_append = pd.merge(Final_Rock_append,data_process_result,how='inner',on=['Sampling Task ID'])

Final_Rock_append['Object ID'] = "id=" + Final_Rock_append['Object ID'].astype(str)
Final_Rock_append['Sampling Task ID'] = "id=" + Final_Rock_append['Sampling Task ID'].astype('int64').astype(str)
Final_Rock_append['Moderation Queue ID'] = "id=" + Final_Rock_append['Moderation Queue ID'].astype('int64').astype(str)

In [140]:
def convert_to_number(df, mod_number, qa_number):
    priority = {
        "remove": 3,
        "selfvisible": 2,
        "normal": 1,
        "Approve": 0,
        "Not Approve": 0,
        "---":0
    }

    df['mod_convert_number'] = 0
    df['qa_convert_number'] = 0

    if df['Sampling Queue'].isin(["QA VN LL Product Comment", "QA VN LL Product Comment Report"]).any():
        df['mod_convert_number'] = df[mod_number].apply(lambda x: int(''.join(str(priority[v]) for v in x.split(','))))
        df['qa_convert_number'] = df[qa_number].apply(lambda x: int(''.join(str(priority[v]) for v in x.split(','))))

    return df

def add_zeros(x, max_length):
    x = str(x)
    while len(x) < max_length:
        x += '0'
    return int(x)

def add_zeros_to_shorter(df):
    max_length = max(df['mod_convert_number'].astype(str).apply(len).max(), df['qa_convert_number'].astype(str).apply(len).max())
    df['mod_convert_number'] = df['mod_convert_number'].apply(add_zeros, args=(max_length,))
    df['qa_convert_number'] = df['qa_convert_number'].apply(add_zeros, args=(max_length,))
    return df

Final_Rock_append = convert_to_number(Final_Rock_append, 'Mod Result','QA Result')
Final_Rock_append = add_zeros_to_shorter(Final_Rock_append)

In [141]:
#FN/FP
Final_Rock_append.loc[(Final_Rock_append['QA Result'] == 'Approve') & (Final_Rock_append['Final process result'] == 'Failed appeal'),'Final False Error']="False Positive"
Final_Rock_append.loc[Final_Rock_append['Final False Error'].isnull(),'Final False Error']="False Negative"
Final_Rock_append.loc[(Final_Rock_append['Mod Result'] == 'Approve') & (Final_Rock_append['Final process result'] == 'Appeal successfully'),'Final False Error']="False Positive"
Final_Rock_append.loc[Final_Rock_append['Final False Error'].isnull(),'Final False Error']="False Negative"
Final_Rock_append.loc[(Final_Rock_append['QA Result'] == 'Approve') & (Final_Rock_append['Final process result'] == 'Edge case'),'Final False Error']="False Positive"
Final_Rock_append.loc[Final_Rock_append['Final False Error'].isnull(),'Final False Error']="False Negative"
Final_Rock_append.loc[(Final_Rock_append['qa_convert_number'] != 0)&(Final_Rock_append['mod_convert_number'] > Final_Rock_append['qa_convert_number'])&(Final_Rock_append['Final process result'] == 'Failed appeal'),'Final False Error']="False Positive"
Final_Rock_append.loc[(Final_Rock_append['qa_convert_number'] != 0)&(Final_Rock_append['mod_convert_number'] < Final_Rock_append['qa_convert_number'])&(Final_Rock_append['Final process result'] == 'Failed appeal'),'Final False Error']="False Negative"
Final_Rock_append.loc[(Final_Rock_append['qa_convert_number'] != 0)&(Final_Rock_append['mod_convert_number'] > Final_Rock_append['qa_convert_number'])&(Final_Rock_append['Final process result'] == 'Edge case'),'Final False Error']="False Positive"
Final_Rock_append.loc[(Final_Rock_append['qa_convert_number'] != 0)&(Final_Rock_append['mod_convert_number'] < Final_Rock_append['qa_convert_number'])&(Final_Rock_append['Final process result'] == 'Edge case'),'Final False Error']="False Negative"
Final_Rock_append.loc[(Final_Rock_append['qa_convert_number'] != 0)&(Final_Rock_append['mod_convert_number'] > Final_Rock_append['qa_convert_number'])&(Final_Rock_append['Final process result'] == 'Appeal successfully'),'Final False Error']="False Negative"
Final_Rock_append.loc[(Final_Rock_append['qa_convert_number'] != 0)&(Final_Rock_append['mod_convert_number'] < Final_Rock_append['qa_convert_number'])&(Final_Rock_append['Final process result'] == 'Appeal successfully'),'Final False Error']="False Positive"
Final_Rock_append.loc[Final_Rock_append['process result'] == '---','Final False Error']="---"

In [142]:
Final_Rock_append.columns

Index(['Selected duration', 'Claimant queue', 'Sampling Queue',
       'Operate status', 'Moderation Queue ID', 'Object ID', 'Task id',
       'Sampling Task ID', 'Task link', 'Moderation Round',
       'Initial Moderation Result', 'Latest Operator', 'Last operator role',
       'Specific operation', 'Moderation Time', 'Sampling respondent Result',
       'Moderation Time.1', 'Arbitrator's Result', 'Final Arbitrator's Result',
       'Corrector's result', 'Correct Moderated Results', 'Appealing Result',
       'Error towards', 'the one who is dealing now', 'Existing tags',
       'tagger', 'claimant', 'process result', 'process reason',
       'beginning time', 'Terminal time', 'handling time', 'Validator',
       'process result.1', 'process reason.1', 'beginning time.1',
       'Terminal time.1', 'handling time.1', 'Arbitrator', 'process result.2',
       'process reason.2', 'beginning time.2', 'Terminal time.2',
       'handling time.2', 'Respondent', 'process result.3', 'process re

In [143]:
Final_Rock_append = Final_Rock_append[['Sampling Queue','claimant','Moderation Time','Object ID','Mod Result', 
                   'Mod Reason','Respondent','Moderation Time.1','Sampling Task ID', 'QA Result', 'QA Reason','Diff case','process result',
                   'process reason','Arbitrator','Final process result','Final False Error','Final policy errors','RCA lvl1','RCA lvl2','RCA lvl3','process reason.2',
                   'Final Arbitrator','process result.4','process reason.4','Task id']].sort_values(ascending=True,by='Moderation Time.1')

Export diff case -> NEW 2023

In [144]:
def filter_time( df,  from_date, to_date):

    Final_Rock_append = df[(df['Moderation Time.1'].dt.strftime('%Y-%m-%d') >= from_date) & (df['Moderation Time.1'].dt.strftime('%Y-%m-%d') <= to_date)]
    Final_Rock_append = Final_Rock_append[Final_Rock_append['claimant'].str.contains('@trans-cosmos.com.vn')]
    Final_Rock_append.to_excel(f"e:/tt/qa_daily_automation/Rock case statistic.xlsx", index=False)

    return Final_Rock_append
# Fill in moderation time (Tue -> Today - 1)
Regular_diff = filter_time(Final_Rock_append, Start_Date_Diff_Regular, End_Date_Diff_Regular)